# Reto técnico

## Instrucciones

La siguiente prueba técnica constará de dos partes: una dedicada a la programación y otra enfocada en SQL.

**Primer reto**

Utiliza cualquier lenguaje de programación. Se prefieren Python o Node.js.

- Implementa pruebas unitarias (opcional).

Algunos aspectos que revisaremos del reto son:

- Funcionamiento del software: revisaremos si la aplicación funciona según lo especificado en el reto.
- Limpieza del código: esperamos que el código sea limpio, comprensible y mantenible.


## Programación

### 1.- Conectarse a la API

In [1]:
#Importamos las librerias que vamos a ocupar
import pandas as pd
import requests
import datetime

# URL de la API
url = 'https://api.stackexchange.com/2.2/search?order=desc&sort=activity&intitle=perl&site=stackoverflow'

# Realizar la solicitud GET
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    print('Conexión exitosa')
else:
     print(f'Error: {response.status_code}')

Conexión exitosa


Para hacer el análisis es necesario convertir los datos de la solicitud en un DataFrame.

In [2]:
#Convertimos los datos recibidos en un DataFrame
data = response.json()
respuestas = pd.DataFrame(data)
respuestas.head()

,items,has_more,quota_max,quota_remaining
0,"{'tags': ['perl', 'visual-studio-code', 'modul...",True,300,281
1,"{'tags': ['perl', 'perlguts'], 'owner': {'repu...",True,300,281
2,"{'tags': ['perl', 'visual-studio-code', 'debug...",True,300,281
3,"{'tags': ['perl', 'openssl'], 'owner': {'reput...",True,300,281
4,"{'tags': ['windows', 'batch-file'], 'owner': {...",True,300,281


Vamos a expandir el diccionario contenido en la columna 'items'. Esto nos permitirá utilizar las claves del diccionario como nombres de columnas, y los valores asociados se colocarán en las filas correspondientes.

In [3]:
respuestas = pd.json_normalize(respuestas['items'])
respuestas.head()

,tags,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,...,owner.user_id,owner.user_type,owner.profile_image,owner.display_name,owner.link,closed_date,closed_reason,owner.accept_rate,accepted_answer_id,protected_date
0,"[perl, visual-studio-code, module, windows-10,...",True,2210,2,2,1712424392,1624108164,1.624123e+09,68047087,CC BY-SA 4.0,...,15309170,registered,https://i.stack.imgur.com/yJylN.jpg?s=256&g=1,Rafael Moreira,https://stackoverflow.com/users/15309170/rafae...,NaN,NaN,NaN,NaN,NaN
1,"[perl, perlguts]",False,50,0,3,1712361953,1712351131,1.712362e+09,78282349,CC BY-SA 4.0,...,12283863,registered,https://i.stack.imgur.com/sNtbo.png?s=256&g=1,Rawley Fowler,https://stackoverflow.com/users/12283863/rawle...,NaN,NaN,NaN,NaN,NaN
2,"[perl, visual-studio-code, debugging, vscode-d...",False,44,0,1,1712306349,1712303165,1.712306e+09,78278459,NaN,...,4316166,registered,https://i.stack.imgur.com/jszIp.jpg?s=256&g=1,kos,https://stackoverflow.com/users/4316166/kos,1.712327e+09,Duplicate,50.0,NaN,NaN
3,"[perl, openssl]",True,58,1,0,1712246288,1712224152,NaN,78273013,CC BY-SA 4.0,...,24004419,registered,https://www.gravatar.com/avatar/aaac97d452b685...,Henry Law,https://stackoverflow.com/users/24004419/henry...,NaN,NaN,NaN,78275067.0,NaN
4,"[windows, batch-file]",True,60,1,0,1712245107,1712201654,1.712234e+09,78271530,CC BY-SA 4.0,...,14001914,registered,https://www.gravatar.com/avatar/d8bd35b00c7593...,mak,https://stackoverflow.com/users/14001914/mak,NaN,NaN,NaN,78271822.0,NaN


### 2. Obtener el número de respuestas contestadas y no contestadas

In [4]:
respuestas['is_answered'].value_counts()

is_answered
True     19
False    11
Name: count, dtype: int64

In [5]:
respuestas_contestadas = respuestas['is_answered'].value_counts()[0]
respuestas_no_contestadas = respuestas['is_answered'].value_counts()[1]

### 3. Obtener la respuesta con menor número de vistas


In [6]:
menor_vistas = respuestas['view_count'].min()
respuesta_menor_vistas = respuestas[respuestas['view_count']==menor_vistas]


### 4. Obtener la respuesta más vieja y más actual

In [7]:
#Obtenemos la fecha más vieja
fecha_mas_vieja = respuestas['creation_date'].min()
#Obtenemos la fecha más actual
fecha_mas_actual = respuestas['creation_date'].max()

#convertimos a formato UNIX a uno legible
fecha_mas_vieja_legible = datetime.datetime.fromtimestamp(fecha_mas_vieja).strftime('%Y-%m-%d')
fecha_mas_actual_legible = datetime.datetime.fromtimestamp(fecha_mas_actual).strftime('%Y-%m-%d')

#Aplicamos máscara para filtrar la respuesta

respuesta_mas_vieja = respuestas[respuestas['creation_date']==fecha_mas_vieja]
respuesta_mas_actual = respuestas[respuestas['creation_date']==fecha_mas_actual]


### 5. Obtener la respuesta del owner que tenga una mayor reputación

In [8]:
owner_mayor_reputacion=respuestas['owner.reputation'].max()
respuesta_owner_mayor_reputacion= respuestas[respuestas['owner.reputation']==owner_mayor_reputacion]

### 6. Imprimir en consola del punto 2 al 5

In [9]:
# Respuestas del punto 2
print(f'Respuestas del punto 2\n')
print(f'El número de respuestas contestadas es {respuestas_contestadas}')
print(f'El número de respuestas no contestadas es {respuestas_no_contestadas}\n')

# Respuesta del punto 3
print('Respuesta del punto 3\n')
print('La respuesta con menor número de vistas es:\n')
print(f'{respuesta_menor_vistas["title"]}\n')

# Respuestas del punto 4
print('Respuestas del punto 4\n')
print(f'La respuesta más vieja se hizo el {fecha_mas_vieja_legible} y es:\n')
print(f'{respuesta_mas_vieja["title"]}\n')
print(f'La respuesta más actual se hizo el {fecha_mas_actual_legible} y es:\n')
print(f'{respuesta_mas_actual["title"]}\n')

# Respuesta del punto 5
print('Respuesta del punto 5\n')
print('La respuesta del owner con la mayor reputación es:\n')
print(f'{respuesta_owner_mayor_reputacion["title"]}\n')


Respuestas del punto 2

El número de respuestas contestadas es 19
El número de respuestas no contestadas es 11

Respuesta del punto 3

La respuesta con menor número de vistas es:

19    Webmin CentOS7 AWS backup errors - perl(S3::AW...
Name: title, dtype: object

Respuestas del punto 4

La respuesta más vieja se hizo el 2008-09-23 y es:

13    Comparison of Python and Perl solutions to Wid...
Name: title, dtype: object

La respuesta más actual se hizo el 2024-04-05 y es:

1    How to count the number of memory allocations ...
Name: title, dtype: object

Respuesta del punto 5

La respuesta del owner con la mayor reputación es:

13    Comparison of Python and Perl solutions to Wid...
Name: title, dtype: object

